In [4]:
import pandas as pd
import numpy as np
import json
import os
import re

In [2]:
DATA = 'objects'
dictionary = {}
failure = []
first = True

for folder in os.listdir(DATA):
    if folder[0] != '.':
        for file in os.listdir(DATA + '/' + folder):
            if file[0] != '.':
                
                f = open(DATA + '/' + folder + '/' + file, 'r')
                text = f.read()
                try :
                    text_json = json.loads(text)
                    
                    if first:
                        first = False
                        keys = text_json.keys()
                        
                        for key in keys:
                            dictionary[key] = []
        
                    for key in keys :
                        try:
                            dictionary[key].append(text_json[key])
                        except:
                            dictionary[key].append(np.nan)
                        
                except:
                    failure.append(file)

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame.from_dict(dictionary)

In [51]:
df.head(1)

,objectID,isHighlight,accessionNumber,isPublicDomain,primaryImage,primaryImageSmall,additionalImages,constituents,department,objectName,...,excavation,river,classification,rightsAndReproduction,linkResource,metadataDate,repository,objectURL,tags,time
0,169884.0,False,2009.300.5672,True,https://images.metmuseum.org/CRDImages/ci/orig...,https://images.metmuseum.org/CRDImages/ci/web-...,[],None,Costume Institute,Stole,...,,,,,,2019-03-12T12:27:30.997Z,"Metropolitan Museum of Art, New York, NY",https://www.metmuseum.org/art/collection/searc...,[],1845.0


In [29]:
len(df)

491946

In [30]:
len(failure)

3

In [32]:
df.to_json('data/corpus.json')

In [33]:
df.columns

Index(['objectID', 'isHighlight', 'accessionNumber', 'isPublicDomain',
       'primaryImage', 'primaryImageSmall', 'additionalImages', 'constituents',
       'department', 'objectName', 'title', 'culture', 'period', 'dynasty',
       'reign', 'portfolio', 'artistRole', 'artistPrefix', 'artistDisplayName',
       'artistDisplayBio', 'artistSuffix', 'artistAlphaSort',
       'artistNationality', 'artistBeginDate', 'artistEndDate', 'objectDate',
       'objectBeginDate', 'objectEndDate', 'medium', 'dimensions',
       'creditLine', 'geographyType', 'city', 'state', 'county', 'country',
       'region', 'subregion', 'locale', 'locus', 'excavation', 'river',
       'classification', 'rightsAndReproduction', 'linkResource',
       'metadataDate', 'repository', 'objectURL', 'tags'],
      dtype='object')

# Get time dimension

In [34]:
time = []

for ind, row in df.iterrows():
    
    if (row['objectBeginDate'] != np.nan) & (row['objectEndDate'] != np.nan):
        time.append(np.mean([row['objectBeginDate'], row['objectEndDate']]))
    elif (row['objectBeginDate'] != np.nan):
        time.append(row['objectBeginDate'])
    elif (row['objectEndDate'] != np.nan):
        time.append(row['objectEndDate'])
    else:
        time.append(np.nan)

In [35]:
df['time'] = time

In [36]:
df['time'].value_counts()

 1849.5    12426
 1749.5     8457
 1888.0     6841
 1900.0     6095
-475.0      5602
 1910.0     5291
 1890.0     4881
 1850.0     4736
 1799.5     4036
 1889.0     3978
 1649.5     3578
 1950.0     2936
 1935.0     2893
 1745.0     2756
 1925.0     2659
 1939.0     2639
-1390.0     2634
 1884.5     2621
 1887.0     2600
-415.0      2572
 1875.0     2550
 1549.5     2548
 1750.0     2440
-490.0      2351
 1860.0     2306
 1850.5     2305
 0.0        2204
-500.0      2188
 1933.0     2167
 1800.0     2011
           ...  
 174.0         1
 1241.0        1
 1240.5        1
 981.5         1
 343.0         1
-1555.0        1
 342.0         1
 530.0         1
 1238.5        1
 41.5          1
 1413.0        1
-1062.5        1
 692.0         1
-618.5         1
-309.0         1
-2462.5        1
 1392.5        1
 696.5         1
 609.0         1
-199.0         1
-48.5          1
 938.0         1
-2686.0        1
 387.5         1
 1394.0        1
 958.0         1
 701.5         1
 1394.5       

In [42]:
df = df.dropna(subset = ['time'])

In [ ]:
df = df[~df['time'].isna()]

In [43]:
len(df)

491202

In [44]:
df.to_json('data/corpus.json')

# Tags

In [55]:
df['classification'].value_counts()

Prints                                                124803
                                                       59621
Photographs                                            41682
Drawings                                               30221
Vases                                                  21331
Books                                                  20667
Ceramics                                               14023
Paintings                                              11697
Textiles-Woven                                         11075
Glass                                                   8971
Negatives                                               6155
Textiles-Laces                                          4978
Sculpture                                               4846
Ceramics-Porcelain                                      4323
Textiles-Embroidered                                    4178
Metalwork-Silver                                        4000
Ceramics-Pottery        

In [58]:
tag_count = []

for ind, item in df['tags'].iteritems():
    tag_count.append(len(item))

In [60]:
df['tag_count'] = tag_count

In [62]:
df = df[(df['tag_count'] != 0) | (df['classification'] != '')]

In [63]:
len(df)

457265

In [73]:
themes = []

for ind, row in df[['tags', 'classification']].iterrows():
    
    theme = []
    
    if row['classification'] != '':
        split = re.split('-', row['classification'])
        for classification in split:
            theme.append(classification)
            
    for tag in row['tags']:
        theme.append(tag)
        
    themes.append(theme)

In [75]:
df['themes'] = themes

In [79]:
df.to_json('data/corpus.json')

## Has image

In [5]:
df = pd.read_json('data/corpus.json')

In [6]:
df = df[df['primaryImageSmall'] != '']

In [8]:
len(df)

215309

In [7]:
df.to_json('data/corpus.json')